In [1]:
import cv2
import numpy as np
import pandas as pd
import os
import time
import random
import sys
sys.path.append(r"C:\Users\Administrator\Desktop\utils\utils\venv")
sys.path.append(r"E:\cancer\kfbreader\kfbreader\Kfbreader-win10-python37")
from Data_PreProcess import Data_preprocess as dp
import kfbReader as kr
import json

In [3]:
def json_reset(js):
    def include(roi,pos):
        return (pos["x"] > roi["x"]) and (pos["y"] > roi["y"]) and (pos["x"] + pos["w"] < roi["x"] + roi["w"]) and (pos["y"] + pos["h"] < roi["y"] + roi["h"])
    
    rois = []
    for info in js:
        if info["class"] == "roi":rois.append(info)
    pos = [[] for _ in range(len(rois))]
    
    for info in js:
        if info["class"] == "roi":continue
        flag = True
        for idx,roi in enumerate(rois):
            if include(roi, info):
                pos[idx].append(info)
                flag = False
        if(flag):print("error")
                
    newJs = []
    for idx,roi in enumerate(rois):
        newJs.append(roi)
        newJs += pos[idx]
    return newJs

In [20]:
roi_area_mean

5497.765039003796

In [21]:
pos_nums_per_roi

5.359550561797753

In [4]:
def split(target):
    labelPath = r"E:\cancer\labels"
    targetFold = r"E:\cancer\pos_%s"%target
    savePath = r"E:\cancer\train_data"

    datList = os.listdir(targetFold)
    #datList = ["T2019_53.kfb"]
    groundTruth = pd.DataFrame(columns=["filename", "x", "y", "w", "h"])
    imgInfo = pd.DataFrame(columns=["filename", "w", "h"])
    for dat in datList:
        reader = kr.reader()
        reader.ReadInfo(os.path.join(targetFold, dat),20,True)
        with open(os.path.join(labelPath, dat[:-4] + ".json"),'r') as f:
            label = json.load(f)
        label = json_reset(label)
        count = 0
        for idx,info in enumerate(label):
            if info["class"] == "roi":
                x = info["x"]
                y = info["y"]
                count += 1
                imgname = dat[:-4] + "_" + str(count) + ".jpg"
                img = reader.ReadRoi(x,y,info["w"],info["h"],20)
                imgInfo = imgInfo.append({"filename":imgname, "w":info["w"], "h":info["h"]},ignore_index=True)
                cv2.imwrite(os.path.join(savePath, imgname),img)
            elif info["class"] == "pos":
                if(info["x"]<x):print(dat)
                groundTruth = groundTruth.append({"filename":imgname, "x":info["x"] - x, "y":info["y"] - y, "w":info["w"], "h":info["h"]},ignore_index=True)
    imgInfo["ratio"] = imgInfo["w"]/imgInfo["h"]
    imgInfo.to_csv(os.path.join(r"E:\cancer\imglabel\imginfo", str(target) +"_info.csv"),index=False)

    groundTruth["w"] += groundTruth["x"]
    groundTruth["h"] += groundTruth["y"]
    groundTruth.columns = ["filename","x1","y1","x2","y2"]
    groundTruth.to_csv(os.path.join(r"E:\cancer\imglabel\groundTruth", str(target) +"_gt.csv"),index=False)

In [7]:
#split(9)

In [ ]:
#def split(target):
labelPath = r"E:\cancer\labels"
labelList = os.listdir(labelPath)
roi_area = []
roi_w = []
roi_h = []
pos_nums_per_roi = []
pos_area = []
pos_h, pos_w = [], []
pos_num = 0
for dat in labelList:
    with open(os.path.join(labelPath, dat),'r') as f:
        label = json.load(f)
    label = json_reset(label)
    for idx,info in enumerate(label):
        if info["class"] == "roi":
            roi_area.append(info["w"] * info["h"])
            roi_h.append(info["h"])
            roi_w.append(info["w"])

            if idx>0:
                pos_nums_per_roi.append(pos_num)
            pos_num = 0
        elif info["class"] == "pos":
            pos_area.append(info["w"] * info["h"])
            pos_h.append(info["h"])
            pos_w.append(info["w"])
            pos_num += 1
roi_area_mean = np.sqrt(np.mean(roi_area))
roi_h_mean = np.mean(roi_h)
roi_w_mean = np.mean(roi_w)
pos_area_mean = np.sqrt(np.mean(pos_area))
pos_h_mean = np.mean(pos_h)
pos_w_mean = np.mean(pos_w)
pos_nums_per_roi = np.mean(pos_nums_per_roi)